# 전력사용량 예측 AI 모델 개발
- 건물유형과 시간에 따른 전기사용량 예측 모델

### 01 라이브러리 불러오기

In [7]:
import tqdm as tq
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from datetime import datetime
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

### 02 파일 불러오기

In [8]:
# 건물정보 파일
building = pd.read_csv('data/building_info.csv')

# Train 데이터셋
train = pd.read_csv('data/train.csv')

# Test 데이터셋
test = pd.read_csv('data/test.csv')

# Submission 파일
submission = pd.read_csv('data/submission.csv')

### 03 건물정보 데이터 전처리

In [9]:
# 태양광이 없는 곳이므로 0값처리
building['태양광용량(kW)'] = building['태양광용량(kW)'].replace('-', 0)
building['태양광용량(kW)'] = building['태양광용량(kW)'].astype('float64')

# 대부분 결측치이고, 해당 기능이 없는 것이므로 0값처리
building['ESS저장용량(kWh)'] = building['ESS저장용량(kWh)'].replace('-', 0)
building['ESS저장용량(kWh)'] = building['ESS저장용량(kWh)'].astype('float64')
building['PCS용량(kW)'] = building['PCS용량(kW)'].replace('-', 0)
building['PCS용량(kW)'] = building['PCS용량(kW)'].astype('float64')

# 연면적 : 냉방면적 비율을 적용하여 냉방면적 결측치 처리
building['냉방면적(m2)'][64] = building['연면적(m2)'][64]/1.27
building['냉방면적(m2)'][65] = building['연면적(m2)'][65]/1.27
building['냉방면적(m2)'][67] = building['연면적(m2)'][67]/1.27

C:\Users\NT930QCG\AppData\Local\Temp\ipykernel_16012\1132713054.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building['냉방면적(m2)'][64] = building['연면적(m2)'][64]/1.27
C:\Users\NT930QCG\AppData\Local\Temp\ipykernel_16012\1132713054.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building['냉방면적(m2)'][65] = building['연면적(m2)'][65]/1.27
C:\Users\NT930QCG\AppData\Local\Temp\ipykernel_16012\1132713054.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

### 04 학습 데이터 전처리

In [10]:
# 주말/공휴일 컬럼 만들기
train['일시'] = pd.to_datetime(train['일시'], format = '%Y%m%d %H')
def is_holiday(row):
    if row['일시'].date() in [pd.to_datetime('2022-06-06').date(), pd.to_datetime('2022-08-15').date()]:
        return 1
    elif row['일시'].dayofweek >= 5:  # 토요일(5) 또는 일요일(6)인 경우
        return 1
    else:
        return 0
train['휴일'] = train.apply(is_holiday, axis=1)

# 일시를 년, 월, 일 컬럼으로 분리(train)
train['년'] = train['일시'].dt.year
train['월'] = train['일시'].dt.month
train['일'] = train['일시'].dt.day
train['시'] = train['일시'].dt.hour
train.drop(['num_date_time', '일시', '년'], axis=1, inplace=True)

# 시간 데이터 표준화
train['sin_time'] = np.sin(2*np.pi*train['시']/24)
train['cos_time'] = np.cos(2*np.pi*train['시']/24)

# 공통된 건물번호 컬럼을 기준으로 데이터 병합
train = pd.merge(building, train, on = '건물번호')

In [11]:
# 건물유형 컬럼 처리 숫자로 변경
train['건물유형'] = train['건물유형'].replace('건물기타', 1)
train['건물유형'] = train['건물유형'].replace('공공', 2)
train['건물유형'] = train['건물유형'].replace('대학교', 3)
train['건물유형'] = train['건물유형'].replace('데이터센터', 4)
train['건물유형'] = train['건물유형'].replace('백화점및아울렛', 5)
train['건물유형'] = train['건물유형'].replace('병원', 6)
train['건물유형'] = train['건물유형'].replace('상용', 7)
train['건물유형'] = train['건물유형'].replace('아파트', 8)
train['건물유형'] = train['건물유형'].replace('연구소', 9)
train['건물유형'] = train['건물유형'].replace('지식산업센터', 10)
train['건물유형'] = train['건물유형'].replace('할인마트', 11)
train['건물유형'] = train['건물유형'].replace('호텔및리조트', 12)

In [12]:
# 기상청에서 결측치 처리에 사용되는 interpolate를 이용해 결측치 처리
train['강수량(mm)'] = train['강수량(mm)'].interpolate(method = 'linear')
train['풍속(m/s)'] = train['풍속(m/s)'].interpolate(method = 'linear')
train['습도(%)'] = train['습도(%)'].interpolate(method = 'linear')
train['일조(hr)'] = train['일조(hr)'].interpolate(method = 'linear')
train['일사(MJ/m2)'] = train['일사(MJ/m2)'].interpolate(method = 'linear')

# 파생변수 생성: 불쾌지수/냉방효율/체감온도
train['불쾌지수'] = 1.8 * train['기온(C)'] -0.55 * (1-train['습도(%)'] * (1.8 * train['기온(C)']-26) + 32)
train['냉방효율'] = train['냉방면적(m2)'] / train['연면적(m2)']
train['체감온도'] = 13.12 + 0.6125 * train['기온(C)'] - 13.947 * (train['풍속(m/s)'] ** 0.16) + 0.486 * train['기온(C)'] * (train['풍속(m/s)'] ** 0.16)

# 태양광용량은 유무로 0과 1로 처리
train['태양광용량(kW)'] = train['태양광용량(kW)'].apply(lambda x : 1 if x != 0 else x)
train['태양광용량(kW)'] = train['태양광용량(kW)'].astype('int')


# 기타 미처리 결측치 처리
train['강수량(mm)'].fillna(0, inplace = True)
train['일조(hr)'].fillna(0, inplace = True)
train['일사(MJ/m2)'].fillna(0, inplace = True)

# 필요없거나 영향 없는 컬럼 삭제
train.drop(['ESS저장용량(kWh)', 'PCS용량(kW)', '강수량(mm)', '일조(hr)', '일사(MJ/m2)'], axis=1, inplace=True)

### 05 테스트 데이터 전처리

In [13]:
# 주말/공휴일 컬럼 만들기
test['일시'] = pd.to_datetime(test['일시'], format = '%Y%m%d %H')
def is_holiday(row):
    if row['일시'].date() in [pd.to_datetime('2022-06-06').date(), pd.to_datetime('2022-08-15').date()]:
        return 1
    elif row['일시'].dayofweek >= 5:  # 토요일(5) 또는 일요일(6)인 경우
        return 1
    else:
        return 0
test['휴일'] = test.apply(is_holiday, axis=1)

# 일시를 년, 월, 일 컬럼으로 분리(test)
test['년'] = test['일시'].dt.year
test['월'] = test['일시'].dt.month
test['일'] = test['일시'].dt.day
test['시'] = test['일시'].dt.hour

# test 시 컬럼 전처리
test['sin_time'] = np.sin(2*np.pi*test['시']/24)
test['cos_time'] = np.cos(2*np.pi*test['시']/24)

# 사용하지 않는 컬럼 삭제
test.drop(['num_date_time', '일시', '년'], axis=1, inplace=True)

# 공통된 건물번호 컬럼을 기준으로 데이터 병합
test = pd.merge(building, test, on = '건물번호')

In [14]:
# 건물유형 컬럼 처리 숫자로 변경
test['건물유형'] = test['건물유형'].replace('건물기타', 1)
test['건물유형'] = test['건물유형'].replace('공공', 2)
test['건물유형'] = test['건물유형'].replace('대학교', 3)
test['건물유형'] = test['건물유형'].replace('데이터센터', 4)
test['건물유형'] = test['건물유형'].replace('백화점및아울렛', 5)
test['건물유형'] = test['건물유형'].replace('병원', 6)
test['건물유형'] = test['건물유형'].replace('상용', 7)
test['건물유형'] = test['건물유형'].replace('아파트', 8)
test['건물유형'] = test['건물유형'].replace('연구소', 9)
test['건물유형'] = test['건물유형'].replace('지식산업센터', 10)
test['건물유형'] = test['건물유형'].replace('할인마트', 11)
test['건물유형'] = test['건물유형'].replace('호텔및리조트', 12)

In [15]:
# 파생변수 생성: 불쾌지수/냉방효율
test['불쾌지수'] = 1.8 * test['기온(C)'] -0.55 * (1-test['습도(%)'] * (1.8 * test['기온(C)']-26) + 32)
test['냉방효율'] = test['냉방면적(m2)'] / test['연면적(m2)']
test['체감온도'] = 13.12 + 0.6215 * test['기온(C)'] - 11.37 * (test['풍속(m/s)'] ** 0.16) + 0.3965 * test['풍속(m/s)'] ** 0.16

# 필요없거나 영향 없는 컬럼 삭제
test.drop(['ESS저장용량(kWh)', 'PCS용량(kW)', '강수량(mm)'], axis=1, inplace=True)

# 평가지표(SMAPE) 설정
def smape(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100
scoring = make_scorer(smape, greater_is_better = False)

### 06-1 건물기타 모델링 & 평가

In [16]:
# train, test 데이터 설정
x = train[train['건물유형']==1][['건물번호', '태양광용량(kW)', '불쾌지수', '체감온도', '냉방효율', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==1]['전력소비량(kWh)']

# 학습데이터 내에서 모델 검증을 위한 train, test 분리 -> 모델성능평가
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 500,
     learning_rate = 0.05,
     max_depth = 13,
     min_child_weight = 1,
     subsample = 0.9,
     colsample_bytree = 0.9)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score1 = smape(y_test, pred)
print(smape_score1)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==1][['건물번호', '태양광용량(kW)', '불쾌지수', '체감온도', '냉방효율', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==1]['전력소비량(kWh)']
test_x = test[test['건물유형']==1][['건물번호', '태양광용량(kW)', '불쾌지수', '체감온도' , '냉방효율', 'sin_time', 'cos_time', '휴일', '월', '일']]
model.fit(x, y)
pred1 = model.predict(test_x)

2.662688415306984


### 06-2 공공건물 모델링 & 평가

In [17]:
# train, test 데이터 설정
x = train[train['건물유형']==2][['건물번호', '태양광용량(kW)', '불쾌지수', '체감온도', '냉방효율', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==2]['전력소비량(kWh)']

# 학습데이터 내에서 모델 검증을 위한 train, test 분리 -> 모델성능평가
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 700,
     learning_rate = 0.01,
     max_depth = 15,
     min_child_weight = 2,
     subsample = 0.9,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score2 = smape(y_test, pred)
print(smape_score2)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==2][['건물번호', '태양광용량(kW)', '불쾌지수', '체감온도', '냉방효율', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==2]['전력소비량(kWh)']
test_x = test[test['건물유형']==2][['건물번호', '태양광용량(kW)', '불쾌지수', '체감온도' , '냉방효율', 'sin_time', 'cos_time', '휴일', '월', '일']]
model.fit(x, y)
pred2 = model.predict(test_x)

1.6936339551389998


### 06-3 대학교 모델링 & 평가

In [18]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==3][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==3]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 1500,
     learning_rate = 0.01,
     max_depth = 13,
     min_child_weight = 3,
     subsample = 0.9,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score3 = smape(y_test, pred)
print(smape_score3)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==3][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==3]['전력소비량(kWh)']
test_x = test[test['건물유형']==3][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일',  '월', '일']]
model.fit(x, y)
pred3 = model.predict(test_x)

1.3062852682607045


### 06-4 데이터센터 모델링 & 평가

In [19]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==4][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==4]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = RandomForestRegressor()
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score4 = smape(y_test, pred)
print(smape_score4)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==4][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==4]['전력소비량(kWh)']
test_x = test[test['건물유형']==4][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일',  '월', '일']]
model.fit(x, y)
pred4 = model.predict(test_x)

0.18145635851074404


### 06-5 백화점및아울렛 모델링 & 평가

In [20]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==5][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==5]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 1500,
     learning_rate = 0.01,
     max_depth = 13,
     min_child_weight = 3,
     subsample = 0.9,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score5 = smape(y_test, pred)
print(smape_score5)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==5][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==5]['전력소비량(kWh)']
test_x = test[test['건물유형']==5][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
model.fit(x, y)
pred5 = model.predict(test_x)

2.447173162817421


### 06-6 병원 모델링 & 평가

In [21]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==6][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==6]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 700,
     learning_rate = 0.05,
     max_depth = 19,
     min_child_weight = 3,
     subsample = 0.8,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score6 = smape(y_test, pred)
print(smape_score6)


# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==6][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==6]['전력소비량(kWh)']
test_x = test[test['건물유형']==6][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일',  '월', '일']]
model.fit(x, y)
pred6 = model.predict(test_x)

1.1255270820020769


### 06-7 상용 모델링 & 평가

In [22]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==7][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==7]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 1500,
     learning_rate = 0.03,
     max_depth = 17,
     min_child_weight = 2,
     subsample = 1.0,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score7 = smape(y_test, pred)
print(smape_score7)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==7][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==7]['전력소비량(kWh)']
test_x = test[test['건물유형']==7][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일',  '월', '일']]
model.fit(x, y)
pred7 = model.predict(test_x)

2.1957155886744197


### 06-8 아파트 모델링 & 평가

In [23]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==8][['건물번호', '냉방효율', 'sin_time', 'cos_time', '휴일', '월', '일', '기온(C)', '풍속(m/s)', '습도(%)']]
y = train[train['건물유형']==8]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 700,
     learning_rate = 0.05,
     max_depth = 19,
     min_child_weight = 3,
     subsample = 0.8,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score8 = smape(y_test, pred)
print(smape_score8)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==8][['건물번호', '냉방효율', 'sin_time', 'cos_time', '휴일', '월', '일', '기온(C)', '풍속(m/s)', '습도(%)']]
y = train[train['건물유형']==8]['전력소비량(kWh)']
test_x = test[test['건물유형']==8][['건물번호', '냉방효율', 'sin_time', 'cos_time', '휴일',  '월', '일', '기온(C)', '풍속(m/s)', '습도(%)']]
model.fit(x, y)
pred8 = model.predict(test_x)

1.5008256459603424


### 06-9 연구소 모델링 & 평가

In [24]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==9][['건물번호', '태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==9]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 700,
     learning_rate = 0.01,
     max_depth = 15,
     min_child_weight = 2,
     subsample = 0.9,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score9 = smape(y_test, pred)
print(smape_score9)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==9][['건물번호', '태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==9]['전력소비량(kWh)']
test_x = test[test['건물유형']==9][['건물번호', '태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일',  '월', '일']]
model.fit(x, y)
pred9 = model.predict(test_x)

1.452246401378051


### 06-10 지식산업센터 모델링 & 평가

In [25]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==10][['건물번호', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==10]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 700,
     learning_rate = 0.05,
     max_depth = 19,
     min_child_weight = 3,
     subsample = 0.8,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score10 = smape(y_test, pred)
print(smape_score10)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==10][['건물번호', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==10]['전력소비량(kWh)']
test_x = test[test['건물유형']==10][['건물번호', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일',  '월', '일']]
model.fit(x, y)
pred10 = model.predict(test_x)

1.3856469966120302


### 07-11 할인마트 모델링 & 평가

In [26]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==11][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==11]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 1500,
     learning_rate = 0.01,
     max_depth = 13,
     min_child_weight = 3,
     subsample = 0.9,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score11 = smape(y_test, pred)
print(smape_score11)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==11][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==11]['전력소비량(kWh)']
test_x = test[test['건물유형']==11][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일',  '월', '일']]
model.fit(x, y)
pred11 = model.predict(test_x)

2.425239690477601


### 07-12 호텔및리조트 모델링 & 평가

In [27]:
# 독립변수와 종속변수 설정
x = train[train['건물유형']==12][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==12]['전력소비량(kWh)']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 100)

# 모델 종류 선정
model = xgb.XGBRegressor(
     n_estimators = 700,
     learning_rate = 0.05,
     max_depth = 19,
     min_child_weight = 3,
     subsample = 0.8,
     colsample_bytree = 1.0)

# 테스트 데이터에 적용
model.fit(x_train, y_train)
pred = model.predict(x_test)
smape_score12 = smape(y_test, pred)
print(smape_score12)

# 실제 test 데이터에 모델 적용
x = train[train['건물유형']==12][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
y = train[train['건물유형']==12]['전력소비량(kWh)']
test_x = test[test['건물유형']==12][['태양광용량(kW)', '불쾌지수', '냉방효율', '체감온도', 'sin_time', 'cos_time', '휴일', '월', '일']]
model.fit(x, y)
pred12 = model.predict(test_x)

2.621739799305645


### 08 Submission 파일에 예측값 넣기

In [28]:
submission['answer'][0:2520] = pred1
submission['answer'][2520:3864] = pred2
submission['answer'][3864:5208] = pred3
submission['answer'][5208:6048] = pred4
submission['answer'][6048:7392] = pred5
submission['answer'][7392:8736] = pred6
submission['answer'][8736:10080] = pred7
submission['answer'][10080:11424] = pred8
submission['answer'][11424:12768] = pred9
submission['answer'][12768:14112] = pred10
submission['answer'][14112:15456] = pred11
submission['answer'][15456:16800] = pred12
submission.to_csv('submission7-3.csv', index = False)

C:\Users\NT930QCG\AppData\Local\Temp\ipykernel_16012\3739484150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['answer'][0:2520] = pred1


In [29]:
print(f'1번건물 Smape Score: {smape_score1}')
print(f'2번건물 Smape Score: {smape_score2}')
print(f'3번건물 Smape Score: {smape_score3}')
print(f'4번건물 Smape Score: {smape_score4}')
print(f'5번건물 Smape Score: {smape_score5}')
print(f'6번건물 Smape Score: {smape_score6}')
print(f'7번건물 Smape Score: {smape_score7}')
print(f'8번건물 Smape Score: {smape_score8}')
print(f'9번건물 Smape Score: {smape_score9}')
print(f'10번건물 Smape Score: {smape_score10}')
print(f'11번건물 Smape Score: {smape_score11}')
print(f'12번건물 Smape Score: {smape_score12}')

1번건물 Smape Score: 2.662688415306984
2번건물 Smape Score: 1.6936339551389998
3번건물 Smape Score: 1.3062852682607045
4번건물 Smape Score: 0.18145635851074404
5번건물 Smape Score: 2.447173162817421
6번건물 Smape Score: 1.1255270820020769
7번건물 Smape Score: 2.1957155886744197
8번건물 Smape Score: 1.5008256459603424
9번건물 Smape Score: 1.452246401378051
10번건물 Smape Score: 1.3856469966120302
11번건물 Smape Score: 2.425239690477601
12번건물 Smape Score: 2.621739799305645
